## Intro

Rust takes a lot of inspiration from functional languages. <span style="color:skyblue">*Programming in a functional style often includes using functions as values by passing them in arguments, returning them from other functions, assigning them to variables for later execution, and so forth.*</span>

In this chapter, we will cover:

- **Closures**, a function-like construct you can store in a variable
- **Iterators**, a way of processing a series of elements
- How to use closures and iterators to improve the I/O project in Chapter 12
- The performance of closures and iterators (Spoiler alert: they're faster than you might think!)


## Closures: Anonymous Functions that Capture Their Environment

<span style="color:skyblue">***Rust’s closures are anonymous functions you can save in a variable or pass as arguments to other functions. You can create the closure in one place and then call the closure elsewhere to evaluate it in a different context. Unlike functions, closures can capture values from the scope in which they’re defined. We’ll demonstrate how these closure features allow for code reuse and behavior customization.***</span>

### Capturing the Environment with Closures
*Scenario*: Every so often, our t-shirt company gives away an exclusive, limited-edition shirt to someone on our mailing list as a promotion. People on the mailing list can optionally add their favorite color to their profile. If the person chosen for a free shirt has their favorite color set, they get that color shirt. If the person hasn’t specified a favorite color, they get whatever color the company currently has the most of.

In [2]:
#[derive(Debug, PartialEq, Copy, Clone)]
enum ShirtColor {
    Red,
    Blue,
}

/// representing the shirt colors currently in stock. 
struct Inventory {
    shirts: Vec<ShirtColor>,
}

impl Inventory {
    /// gets the optional shirt color preference of the free shirt winner, 
    /// and returns the shirt color the person will get
    fn giveaway(&self, user_preference: Option<ShirtColor>) -> ShirtColor {
        user_preference.unwrap_or_else(|| self.most_stocked())
        // here, we used closure. We got the user preference as a 
        // parameter of type `Option<ShirtColor>` and call the 
        // `unwrap_or_else` method on `user_preference`. The
        // `unwrap_or_else` takes one argument: a closure without
        // any arguments, calls `self.most_stocked()` in the body 
        // that returns a value `T` of the calling `Option<T>`
        // (In this case, `T` is `ShirtColor`)
        // If the `Option<T>` is the `Some` variant, `unwrap_or_else` 
        // returns the value from within the `Some`. If the `Option<T>` 
        // is the `None` variant, `unwrap_or_else` calls the closure 
        // and returns the value returned by the closure.
        // Note: Here, the closure captures an immutable reference 
        // to the self `Inventory` instance and passes it with the 
        // code we specify to the `unwrap_or_else` method. 
        // Functions, on the other hand, are not able to capture 
        // their environment in this way.
    }

    /// find out if the store has more blue / red shirts in the stock
    fn most_stocked(&self) -> ShirtColor {
        let mut num_red = 0;
        let mut num_blue = 0;

        for color in &self.shirts {
            match color {
                ShirtColor::Red => num_red += 1,
                ShirtColor::Blue => num_blue += 1,
            }
        }

        if num_red > num_blue {
            ShirtColor::Red
        } else {
            ShirtColor::Blue
        }
    }
}

fn main() {
    let store = Inventory {
        shirts: vec![ShirtColor::Blue, ShirtColor::Red, ShirtColor::Blue],
    };

    let user_pref1 = Some(ShirtColor::Red);
    let giveaway1 = store.giveaway(user_pref1);
    println!(
        "The user with preference {:?} gets {:?}",
        user_pref1, giveaway1
    );

    let user_pref2 = None;
    let giveaway2 = store.giveaway(user_pref2);
    println!(
        "The user with preference {:?} gets {:?}",
        user_pref2, giveaway2
    );
}

main()

The user with preference Some(Red) gets Red
The user with preference None gets Blue


()

### Closure Type Inference and Annotation

Unlike functions, closures don’t usually require you to annotate the types of the parameters or the return value like `fn` functions do. This is because functions are often exposed to users so we need everyone to agree on the types of values; closures, on other other hand, are used without naming and are not exposed to users.

As with variables, we can add type annotations if we want to increase explicitness and clarity at the cost of being more verbose than is strictly necessary.

In [4]:
use std::thread;
use std::time::Duration;

fn generate_workout(intensity: u32, random_number: u32) {
    let expensive_closure = |num: u32| -> u32 {  // type annotation with
                                                 // closure here
        println!("calculating slowly...");
        thread::sleep(Duration::from_secs(2));
        num
    };

    if intensity < 25 {
        println!("Today, do {} pushups!", expensive_closure(intensity));
        println!("Next, do {} situps!", expensive_closure(intensity));
    } else {
        if random_number == 3 {
            println!("Take a break today! Remember to stay hydrated!");
        } else {
            println!(
                "Today, run for {} minutes!",
                expensive_closure(intensity)
            );
        }
    }
}

fn main() {
    let simulated_user_specified_value = 10;
    let simulated_random_number = 7;

    generate_workout(simulated_user_specified_value, simulated_random_number);
}

main()

calculating slowly...
Today, do 10 pushups!
calculating slowly...
Next, do 10 situps!


()